In [79]:
import torch
import numpy as np
from PIL import Image
from pprint import pprint
import matplotlib.pyplot as plt
from mmdet.models.backbones import ResNet
from mmdet.models.flow_heads import FlowHead
from torchvision.transforms.functional import resize

# test image
img0 = Image.open('/external/datasets/kitti/2015/training/image_2/000000_10.png')
img0 = np.asarray(resize(img0, (256, 832)))
img0 = torch.from_numpy(img0).permute(2, 0, 1).unsqueeze(0).cuda() / 255. 

img1 = Image.open('/external/datasets/kitti/2015/training/image_2/000000_11.png')
img1 = np.asarray(resize(img1, (256, 832)))
img1 = torch.from_numpy(img1).permute(2, 0, 1).unsqueeze(0).cuda() / 255. 

In [81]:
backbone = ResNet(depth=50).cuda()
features0 = [f.contiguous() for f in backbone(img0)]
features1 = [f.contiguous() for f in backbone(img1)]
pprint([f.shape for f in features0])

[torch.Size([1, 256, 64, 208]),
 torch.Size([1, 512, 32, 104]),
 torch.Size([1, 1024, 16, 52]),
 torch.Size([1, 2048, 8, 26])]


In [58]:
from spatial_correlation_sampler import SpatialCorrelationSampler
corr = SpatialCorrelationSampler(kernel_size=1, patch_size=9, stride=1, padding=0, dilation=1, dilation_patch=1)
corr0 = corr(features0[0], features1[0])